In [1]:
import pandas as pd
import re
import threading
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException, TimeoutException

def scrape_products():
    # Configura el path del ChromeDriver
    service = Service(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\SELENIUM\chromedriver.exe')
    driver = webdriver.Chrome(service=service)

    # Lista para almacenar los datos de todos los productos
    all_products_data = []

    # Fecha actual en formato DD-MM-YYYY
    today_date = datetime.now().strftime('%d-%m-%Y')

    # Tallas a extraer
    sizes = ['20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40']

    driver.get("https://www.skechers.cl/kids")

    # Espera a que carguen los elementos
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.item-producto a')))
    except TimeoutException:
        print("No se encontraron enlaces de productos en el tiempo esperado.")
        driver.quit()
        return

    start_time = time.time()  # Guardar el tiempo de inicio

    # Itera hasta que no queden más productos o hayan pasado 5 minutos
    while True:
        if time.time() - start_time > 60 * 60:  # 5 minutos
            print("Se ha alcanzado el límite de tiempo de 5 minutos.")
            break

        product_links = driver.find_elements(By.CSS_SELECTOR, '.item-producto a')

        # Si no hay más enlaces, salir
        if not product_links:
            print("No hay más productos para procesar.")
            break

        # Se guardan los enlaces procesados
        processed_links = set()

        for index in range(len(product_links)):
            product_link = product_links[index]
            product_url = product_link.get_attribute('href')
            if product_url in processed_links:
                continue  # Salta si ya se procesó este enlace

            try:
                # Hacer clic directamente en el enlace del producto
                driver.execute_script("arguments[0].scrollIntoView(true);", product_link)
                time.sleep(1)  # Reducción del tiempo de espera
                product_link.click()

                # Espera a que cargue la página del producto
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))

                # Extrae el precio original
                try:
                    original_price_element = driver.find_element(By.XPATH, "//s")
                    original_price_text = original_price_element.text
                    original_price = original_price_text.replace('$', '').replace('.', '').strip()
                except NoSuchElementException:
                    original_price = ''  # O puedes usar '0'

                # Extrae el precio final
                try:
                    final_price = driver.find_element(By.CSS_SELECTOR, 'b.text-danger').text
                except NoSuchElementException:
                    final_price = None

                # Extrae el SKU
                try:
                    sku_element = driver.find_element(By.CSS_SELECTOR, 'p.nomargin.text-muted')
                    sku_text = sku_element.text
                    sku = sku_text.split("Código: ")[1].strip()  # Extraer el SKU después de "Código: "
                except NoSuchElementException:
                    sku = None

                # Inicializa listas para tallas y stock
                stock_dict = {size: 0 for size in sizes}

                for size in sizes:
                    try:
                        size_link = driver.find_element(By.CSS_SELECTOR, f'a[data-talla="{size}"]')
                        stock_quantity = int(size_link.get_attribute('data-cantidad'))
                        stock_dict[size] = stock_quantity
                    except (NoSuchElementException, StaleElementReferenceException):
                        stock_dict[size] = 0

                # Extrae la URL de la imagen
                try:
                    # Busca la imagen dentro del enlace <a> con la clase 'thumbnail active' y extrae el src
                    image_element = driver.find_element(By.XPATH, "//a[@class='thumbnail active']/img")
                    image_url = image_element.get_attribute('src')  # Extrae la URL de la imagen
                    
                    # Limpiar la URL si es necesario (por ejemplo, eliminar //limonada o ajustar el nombre de la imagen)
                    image_url_cleaned = re.sub(r'//limonada', '', image_url)  # Eliminar partes no deseadas
                    image_url_final = re.sub(r'_(\d+)\.jpg$', '_1.jpg', image_url_cleaned)  # Cambiar el nombre si es necesario

                except NoSuchElementException:
                    image_url_final = ''  # Si no se encuentra la imagen, asigna un valor vacío

                # Extrae la URL de la página del producto
                product_url = driver.current_url
                base_url = "https://www.skechers.cl/detalle/"
                extracted_part = product_url.replace(base_url, "")

                # Limpiar el nombre de guiones por espacios y eliminar "Skechers"
                product_name = extracted_part.replace('-', ' ').title()
                product_name = ' '.join(word for word in product_name.split() if word.lower() != 'skechers')

                # Extrae el color
                try:
                    elemento = driver.find_element(By.CSS_SELECTOR, "a.tallas.thumbnail[data-color]")
                    color = elemento.get_attribute('data-color')
                except NoSuchElementException:
                    color = ''  # Si no se encuentra el color, asigna un valor vacío

                # Agrega los datos del producto a la lista
                product_data = {
                    'Nombre': product_name,
                    'Marca': 'Skechers',
                    'Color': color,  # Columna Color en blanco
                    'SKU': sku,
                    'Precio Original': original_price if original_price else '0',  # Usar '0' si no hay precio
                    'Precio Final': final_price,
                    'URL de la Imagen': image_url_final,
                    'URL Página': product_url,
                    'Fecha': today_date,
                }
                product_data.update(stock_dict)
                all_products_data.append(product_data)

                # Guarda los datos en Excel
                df = pd.DataFrame(all_products_data)
                df.to_excel(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\CALZADO.xlsx', index=False)

                # Añadir enlace a los procesados
                processed_links.add(product_url)

            except (ElementClickInterceptedException, StaleElementReferenceException):
                print(f"Error: Elemento bloqueado o obsoleto para hacer clic en el producto {product_url}. Reintentando...")
                driver.back()
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.item-producto a')))
                product_links = driver.find_elements(By.CSS_SELECTOR, '.item-producto a')  # Refresca la lista de productos
                continue  # Intenta nuevamente con el mismo enlace

            # Regresa a la página anterior
            driver.back()

            # Espera a que cargue la página de productos nuevamente
            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.item-producto a')))
            product_links = driver.find_elements(By.CSS_SELECTOR, '.item-producto a')  # Refresca la lista de productos

    # Cierra el navegador
    driver.quit()

# Iniciar el hilo para ejecutar el scraping en segundo plano
scraping_thread = threading.Thread(target=scrape_products)
scraping_thread.start()

# Puedes continuar con otras tareas aquí...
print("El scraping se está ejecutando en segundo plano. Puedes hacer otras tareas.")


El scraping se está ejecutando en segundo plano. Puedes hacer otras tareas.


Exception in thread Thread-5 (scrape_products):
Traceback (most recent call last):
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\constanza.perez\AppData\Local\Temp\ipykernel_16408\1920419425.py", line 165, in scrape_products
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\selenium\webdriver\support\wait.py", line 96, in until
    value = method(self._driver)
            ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\selenium\webdriver\support\expected_conditions.py", line 191, in _predicate
    return driver.find_elements(*locator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.